In [5]:
import os
import re
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
# Путь к папке с атаками
base_path = "./attacks/"
attack_folders = os.listdir(base_path)
attack_labels = {attack: idx + 1 for idx, attack in enumerate(attack_folders)}
attack_labels

{'brute-http': 1,
 'brute-ftp': 2,
 'inf-usb': 3,
 'xss': 4,
 'brute-ssh': 5,
 'inf-dropbox': 6,
 'sql-inj': 7,
 'botnet-ares': 8}

In [ ]:
# Найти все уникальные размеры выборок
packet_intervals = set()

for attack in attack_folders:
    common_path = os.path.join(base_path, attack, "csvs")
    for traffic_type in ["normal", "attack"]:
        traffic_path = os.path.join(common_path, traffic_type)
        for filename in os.listdir(traffic_path):
            # Извлечение размера выборки из имени файла с помощью регулярного выражения
            match = re.search(r"_(\d+)_\d+_", filename)
            if match:
                packet_intervals.add(int(match.group(1)))

# Сортируем размеры выборок
packet_intervals = sorted(packet_intervals)
print("Найденные размеры выборок пакетов:", packet_intervals)

Найденные размеры выборок пакетов: [10, 50, 100, 250, 500, 750, 1000, 5000, 10000]


In [ ]:
# Объединение данных для каждого размера выборки
for packet_size in packet_intervals:
    combined_df = pd.DataFrame()

    for attack in attack_folders:
        common_path = os.path.join(base_path, attack, "csvs")

        # Путь к файлам нормального и атакующего трафика для текущей длины выборки
        normal_files = [
            f
            for f in os.listdir(os.path.join(common_path, "normal"))
            if f"_{packet_size}_0_" in f
        ]
        attack_files = [
            f
            for f in os.listdir(os.path.join(common_path, "attack"))
            if f"_{packet_size}_1_" in f
        ]

        # Чтение и добавление меток
        for file in normal_files:
            normal_df = pd.read_csv(os.path.join(common_path, "normal", file))
            normal_df["Label"] = 0  # Метка для нормального трафика
            combined_df = pd.concat([combined_df, normal_df], ignore_index=True)

        for file in attack_files:
            attack_df = pd.read_csv(os.path.join(common_path, "attack", file))
            attack_df["Label"] = attack_labels[attack]  # Уникальная метка для атаки
            combined_df = pd.concat([combined_df, attack_df], ignore_index=True)

    # Сохранение объединенного CSV для текущего размера выборки
    combined_file = f"combined_{packet_size}_packets.csv"
    combined_df.to_csv(combined_file, index=False)

    # Построение корреляционной матрицы
    corr_matrix = combined_df.corr()
    plt.figure(figsize=(12, 8))
    sns.heatmap(
        corr_matrix,
        annot=True,
        fmt=".2f",
        cmap="coolwarm",
        cbar_kws={"label": "Correlation Coefficient"},
    )
    plt.title(f"Correlation Matrix for {packet_size} Packet Interval")
    plt.show()